In [1]:
using QuantumOptics
using PyPlot
using SpecialFunctions

In [12]:
global suma
################
#plt.rc("text", usetex=true)
#PyCall.PyDict(plt."rcParams")["text.latex.preamble"] = [raw"\usepackage{amsmath}"]
#rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
#font0 = Dict(
#            "font.size" => 16,
##        "axes.labelweight" => "bold",
#            "axes.labelsize" => 16,
#            "xtick.labelsize" => 13,
#            "ytick.labelsize" => 13,
#            "legend.fontsize" => 16,
#)
# merge!(rcParams, font0)
#############################################################
# Definiendo los parámetros
Escala = 1;
ωb = 10
ωa =  ωb / 2
Ep = 0.9
g = 0.1
l = 1
l1=0
suma=0;
suma1=0;
suma2=0;
while l <= 10
           suma1 =  suma1 + (besselj(l, Ep)^2) / ((ωa+ωb)^2-(l^2)*(2*ωb)^2)
           suma2 =  suma2 + (besselj(l, Ep)^2) / ((ωa-ωb)^2-(l^2)*(2*ωb)^2)
           global l += 1
           global suma1
           global suma2
end
Nu = 2 * (ωb-((g^2) * ωa * (besselj(0, Ep))^2)/(ωa^2-ωb^2) - (ωa+ωb)*(g^2)*suma1 - (ωa-ωb)*(g^2)*suma2)
gama = Ep * Nu / ωa
while l1 <= 20
           suma =  suma + ((2*l1+1) * besselj(l1, Ep) * besselj(l1 + 1, Ep)) / (ωa^2 - ωb^2*(2*l1+1)^2)
           #suma =  suma + (besselj(l1, Ep) * besselj(l1 + 1, Ep)) / (ωa^2-(ωb + l1 * Nu)^2)
           global l1 += 1
           global suma
end
geff = -2 * (g^2) * ωb * suma
# Definiendo las bases para la cavidad(Fock)
bc = FockBasis(10)
# Construcción de operadores
a = destroy(bc) ⊗ one(bc)
b =  one(bc) ⊗ destroy(bc)
na = number(bc) ⊗ one(bc)
nb =  one(bc) ⊗ number(bc)
# Definición del estado inicial
ψ₀ =  fockstate(bc::FockBasis, 0) ⊗ fockstate(bc::FockBasis, 0)
# = coherentstate(bc, α) ⊗ coherentstate(bc, α)
ψ1 =  fockstate(bc::FockBasis, 2) ⊗ fockstate(bc::FockBasis, 0)
# = coherentstate(bc, 2) ⊗ coherentstate(bc, α)
# Defición de lista de pasos temporales
T = [0:0.1:400;]
# Componentes del Hamiltoniano
 H0 = ωb * nb
 H1 = g * (a + dagger(a)) * (b + dagger(b))
 function Hₜ(t, ρ) # time-dependent Hamiltonian
   H = ωa * ( 1 + gama * cos(Nu*t)) * na + H0 + H1
   return H
end
 tout, ψₜ = timeevolution.schroedinger_dynamic(T,ψ₀, Hₜ)
 # ψₜ = Escala * ψₜ
 NFot = real(expect(nb,  ψₜ))
                #AA = projector(ψ1::Ket)
                 #ProbA = real(expect(AA, ψₜ))
                   #Prob02 = sum(ProbA) /(Escala * length(ProbA))
#figure()
plot(10*geff * T,NFot)
plot(10*geff * T,(Escala * (sinh.(-2*geff*T)).^2),"k--")
#xlabel(L"$g_{eff}t$")
#ylabel(L"\langle n_{b} \rangle")
#savefig("ImageAP2021.eps")
#close()


Figure(PyObject <Figure size 640x480 with 1 Axes>)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x0000000000E6C430>